# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
'''
Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
sessionId = 338, and itemInSession = 4
'''
query = "CREATE TABLE IF NOT EXISTS song_info_by_session "
query += "(sessionId int, itemInSession int, artist_name text, song_title text, song_length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# Insert relevant Data from event_datafile_new.csv to song_info_by_session_id table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_session(sessionId, itemInSession, artist_name, song_title, song_length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Testing the query to get the artist_name, song_title, song_length where sessionId=338 AND itemInSession=4
query = "SELECT artist_name, song_title, song_length FROM song_info_by_session WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
x = PrettyTable()
for row in rows:
    x.field_names = ["Artist Name", "Song Title", "Song Length"]
    x.add_row([row.artist_name, row.song_title, row.song_length])

print(x)

+-------------+---------------------------------+--------------------+
| Artist Name |            Song Title           |    Song Length     |
+-------------+---------------------------------+--------------------+
|  Faithless  | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-------------+---------------------------------+--------------------+


In [11]:
'''
Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
for userid = 10, sessionid = 182
'''
query = "CREATE TABLE IF NOT EXISTS artist_song_info_by_user "
query += "(userId int, sessionId int, itemInSession int, artist_name text, song_title text, firstName text, lastName text, \
PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# Insert relevant Data from event_datafile_new.csv to artist_song_info_by_user table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_info_by_user(userId, sessionId, itemInSession, artist_name, song_title, firstName, lastName)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)

In [13]:
## Testing the query to get the artist_name, song_title, user(firstname and lastname) where userID=10 AND sessionId=182
query = "SELECT artist_name, song_title, firstName, lastName FROM artist_song_info_by_user WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

x=PrettyTable()
for row in rows:
#     print(row)
    x.field_names = ['Artist Name','Song Title', 'First Name', 'Last Name']
    x.add_row([row.artist_name, row.song_title, row.firstname, row.lastname])
print(x)

+-------------------+------------------------------------------------------+------------+-----------+
|    Artist Name    |                      Song Title                      | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


In [14]:
pd.set_option('display.max_colwidth',100) 

In [15]:
## Testing the query to get the artist_name, song_title, user(firstname and lastname) where userID=10 AND sessionId=182 
## Using Data Frame
query = "SELECT artist_name, song_title, firstName, lastName FROM artist_song_info_by_user WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

result_df = pd.DataFrame(rows._current_rows,columns =['Artist Name','Song Title', 'First Name', 'Last Name'])
result_df


,Artist Name,Song Title,First Name,Last Name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio Edit),Sylvie,Cruz


In [16]:
'''
Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
'''
query = "CREATE TABLE IF NOT EXISTS user_by_song "
query += "(song_title text, userId int, firstName text, lastName text, PRIMARY KEY (song_title, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)         

In [17]:
# Insert relevant Data from event_datafile_new.csv to user_by_song table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_song(song_title, userId, firstName, lastName)"
        query = query + " VALUES(%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

In [18]:
## Testing the query to get the userId, user(firstname and lastname) where song_title='All Hands Against His Own'
query = "SELECT userId, firstName, lastName FROM user_by_song WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

x = PrettyTable()
for row in rows:
    x.field_names = ['User Id','First Name','Last Name']
    x.add_row([row.userid, row.firstname, row.lastname])
print(x)

+---------+------------+-----------+
| User Id | First Name | Last Name |
+---------+------------+-----------+
|    29   | Jacqueline |   Lynch   |
|    80   |   Tegan    |   Levine  |
|    95   |    Sara    |  Johnson  |
+---------+------------+-----------+


### Drop the tables before closing out the sessions

In [19]:
try:
    session.execute("DROP TABLE song_info_by_session")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE artist_song_info_by_user")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE user_by_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()